In [1]:
import os
import pandas as pd

In [3]:
filepath = '/home/shuang/projects/eqtm/data/output/random20k_gt0.5_overlapRatio.txt'
filecontent = pd.read_csv(filepath,sep='\t',index_col=0)
print(filecontent.shape)
filecontent.head()

(20000, 33)


,H4K5ac,H2A.Z,H2BK120ac,H3K79me2,H3K27ac,H2BK20ac,H3K14ac,H3K9ac,H4K20me1,H3K4ac,...,H3K9me1,H3K4me2,H3K27me3,H3K79me1,H2AK9ac,H3T11ph,H4K8ac,H4K91ac,H3K56ac,SNPName
0,0.582677,0.535433,0.511811,0.543307,0.511811,0.173228,0.582677,0.559055,0.000000,0.614173,...,0.0,0.582677,0.000000,0.559055,0.346457,0.464567,0.598425,0.559055,0.527559,cg26964415
1,0.031496,0.000000,0.015748,0.039370,0.094488,0.000000,0.015748,0.023622,0.000000,0.007874,...,0.0,0.062992,0.000000,0.141732,0.000000,0.015748,0.015748,0.000000,0.000000,cg10400499
2,0.314961,0.535433,0.456693,0.070866,0.472441,0.062992,0.551181,0.559055,0.000000,0.574803,...,0.0,0.582677,0.385827,0.212598,0.094488,0.267717,0.590551,0.551181,0.078740,cg27532722
3,0.606299,0.535433,0.535433,0.496063,0.511811,0.165354,0.598425,0.559055,0.322835,0.614173,...,0.0,0.582677,0.015748,0.488189,0.362205,0.527559,0.598425,0.559055,0.582677,cg05322697
4,0.062992,0.354331,0.031496,0.007874,0.133858,0.007874,0.055118,0.141732,0.370079,0.023622,...,0.0,0.496063,0.480315,0.015748,0.047244,0.047244,0.031496,0.062992,0.015748,cg22494858


In [6]:
len(filecontent.SNPName.unique())

19199

In [13]:
# add zscore
zscorepath = '/home/shuang/projects/boxy_eqtm/data/eqtmZscores/fdr_gt0.05/random20k_gt0.5_withTss.txt'
zscore = pd.read_csv(zscorepath,sep=',')
zscore.head()

,PValue,SNPName,SNPChr,SNPChrPos,ProbeName,ProbeChr,ProbeCenterChrPos,CisTrans,SNPType,AlleleAssessed,...,HGNCName,IncludedDatasetsCorrelationCoefficient,Meta-Beta (SE),Beta (SE),FoldChange,FDR,TssSite,chr,TssDistance,checkChr
0,0.530528,cg26964415,19,18668516,ENSG00000130475,19,17878952,cis,C/T,C,...,FCHO1,0.0156593;-0.0489998;0.0015376;0.0566576;0.101...,0E0 (0E0),0.0156593 (0.0425576);-0.0489998 (0.0367414);0...,0;0;0;0;0;0,0.939263,17858527,19,809989,True
1,0.835073,cg10400499,19,53088849,ENSG00000170949,19,53588274,cis,C/T,C,...,ZNF160,0.0301376;0.0303174;-0.0034855;-0.0736521;-0.0...,0E0 (0E0),0.0301376 (0.0425435);0.0303174 (0.0367687);-0...,0;0;0;0;0;0,0.984766,53606689,19,517840,True
2,0.990107,cg27532722,22,24115520,ENSG00000189269,22,23962573,cis,C/T,C,...,C22orf43,0.0358173;-0.0086801;-0.0126815;0.0186243;0.01...,0E0 (0E0),0.0358173 (0.0425355);-0.0086801 (0.0367842);-...,0;0;0;0;0;0,0.999198,23974508,22,141012,True
3,0.673317,cg05322697,16,67694584,ENSG00000102977,16,67693064,cis,C/T,C,...,ACD,0.0688032;-0.0468145;0.0155556;-0.0336869;-0.1...,0E0 (0E0),0.0688032 (0.042462);-0.0468145 (0.0367453);0....,0;0;0;0;0;0,0.964430,67694713,16,129,True
4,0.525689,cg22494858,1,110611712,ENSG00000134202,1,110280469,cis,C/T,C,...,GSTM3,-0.0571232;0.0602986;-0.0213098;0.1202855;-0.0...,0E0 (0E0),-0.0571232 (0.0424933);0.0602986 (0.0367187);-...,0;0;0;0;0;0,0.938146,110284384,1,327328,True


In [16]:
def changeCpGColumnName_addZscoreTss_forRandom(feature_file_path,feature_sep,zscore_path,zscore_sep):
    feature_file = pd.read_csv(feature_file_path, sep=feature_sep,index_col=0)
    feature_file = feature_file.rename(index=str,columns={'SNPName':'cpgName'})
    def read_zscore(zscore_path):
        zscore_file = pd.read_csv(zscore_path,sep=zscore_sep,index_col=0)

        zscore_file['cpgName'] = zscore_file[['SNPName']]
        zscore_dic = zscore_file.set_index('cpgName')['OverallZScore'].to_dict()
        tss_dic = zscore_file.set_index('cpgName')['TssDistance'].to_dict()
        return zscore_dic,tss_dic
    zscore_dic,tss_dic = read_zscore(zscore_path)
    def map_zscore(row):
        if row in zscore_dic:
            return zscore_dic[row]
        return None
    def map_tss(row):
        if row in tss_dic:
            return tss_dic[row]
        return None
    feature_file['zscore'] = feature_file['cpgName'].apply(map_zscore)
    feature_file['TssDistance'] = feature_file['cpgName'].apply(map_tss)
    return feature_file

In [17]:
withZscore = changeCpGColumnName_addZscoreTss_forRandom(filepath, '\t', zscorepath, ',')
withZscore.head()

,H4K5ac,H2A.Z,H2BK120ac,H3K79me2,H3K27ac,H2BK20ac,H3K14ac,H3K9ac,H4K20me1,H3K4ac,...,H3K27me3,H3K79me1,H2AK9ac,H3T11ph,H4K8ac,H4K91ac,H3K56ac,cpgName,zscore,TssDistance
0,0.582677,0.535433,0.511811,0.543307,0.511811,0.173228,0.582677,0.559055,0.000000,0.614173,...,0.000000,0.559055,0.346457,0.464567,0.598425,0.559055,0.527559,cg26964415,0.627393,809989
1,0.031496,0.000000,0.015748,0.039370,0.094488,0.000000,0.015748,0.023622,0.000000,0.007874,...,0.000000,0.141732,0.000000,0.015748,0.015748,0.000000,0.000000,cg10400499,0.208294,517840
2,0.314961,0.535433,0.456693,0.070866,0.472441,0.062992,0.551181,0.559055,0.000000,0.574803,...,0.385827,0.212598,0.094488,0.267717,0.590551,0.551181,0.078740,cg27532722,0.012422,141012
3,0.606299,0.535433,0.535433,0.496063,0.511811,0.165354,0.598425,0.559055,0.322835,0.614173,...,0.015748,0.488189,0.362205,0.527559,0.598425,0.559055,0.582677,cg05322697,0.421439,129
4,0.062992,0.354331,0.031496,0.007874,0.133858,0.007874,0.055118,0.141732,0.370079,0.023622,...,0.480315,0.015748,0.047244,0.047244,0.031496,0.062992,0.015748,cg22494858,0.634617,327328


In [18]:
withZscore.to_csv('/home/shuang/projects/boxy_eqtm/data/dataReadyForModeling/overlapRatioTss/random20k_gt0.5_withZscoreTss.csv')